# Assignment #1: Building an inverted index

Author: **Hicham Mohamad** (hi8826mo-s)

**Natural Language Processing** or **NLP** is a field of Artificial Intelligence that gives the machines the ability to read, understand and derive meaning from human languages.

## Table of Contents
1. [Corpus](#t1)
2. [Indexing one file](#t2)
3. [Reading the content of a folder](#t3)
4. [Creating a master index](#t4)
5. [Representing Documents with tf-idf](#t5)
6. [Comparing Documents](#t6)
7. [Submission](#t7)
8. [Reading](#t8) 

## Objectives

The objectives of this assignment are to:
* Write a program that collects all the words from a set of documents
* Build an index from the words
* Represent a document using the Tf.Idf values
* Write a short report of 1 to 2 pages on the assignment
* Read a short text on an industrial system

## Submission

When you have written all the missing code and run all the cells, *you will submit your notebook to an automatic marking system*. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Description of the assignment

### Outline

In this lab, you will **build an indexer to index all the words in a corpus**. Conceptually, an **index** consists of rows with one word per row and the **list** of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.
<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>

#### Imports

Some imports. Add others as needed

In [1]:
import bz2
import math
import os
import pickle
import regex as re
import requests
import sys
import time
from zipfile import ZipFile

import numpy

## Corpus <a name="t1"/>

You will create an index for a corpus of **Selma Lagerlöf's works**: To gather the corpus, you can alternatively:
1. Download the <a href="https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip">Selma folder</a> and uncompress it. It contains novels by <a href="https://sv.wikipedia.org/wiki/Selma_Lagerl%C3%B6f">Selma Lagerlöf</a>. The text of these novels was extracted from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at <a href="https://litteraturbanken.se/">Litteraturbanken</a>.
2. Or run this cell that will download the corpus and place it in your folder.

In [2]:
# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname) 
    for fname in 
    [
        "bannlyst.txt", 
        "gosta.txt", 
        "herrgard.txt", 
        "jerusalem.txt", 
        "kejsaren.txt", 
        "marbacka.txt", 
        "nils.txt", 
        "osynliga.txt", 
        "troll.txt"
    ]
]

def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""
    
    global SELMA_URL
    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()
        
        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith("__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)
            
        print("Done!")
        
# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")
    
SELMA_FILES

Selma has been downloaded.


['Selma\\bannlyst.txt',
 'Selma\\gosta.txt',
 'Selma\\herrgard.txt',
 'Selma\\jerusalem.txt',
 'Selma\\kejsaren.txt',
 'Selma\\marbacka.txt',
 'Selma\\nils.txt',
 'Selma\\osynliga.txt',
 'Selma\\troll.txt']

### Running the indexer (optional)

In a production context, your final program would take a corpus as input (here the Selma Lagerlöf's novels) and create an index of all the words with their positions. You should be able to run it this way:
<pre>$ python indexer.py folder_name</pre>
In this lab, you will write the index in a Jupyter Notebook. The conversion into a Python program is left as an optional exercise.

## Programming the Indexer

To make programming easier, you will split this exercise into five steps:
1. Index one **file**
2. Read the **content** of a folder
3. Create a **master index** for all the files
4. Use **tfidf** to represent the documents (novels)
5. Compare the documents of a collection

You will use **dictionaries** to represent the **postings**.

### Indexing one file <a name="t2"/>

#### Description

<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
    <li>The index file will contain all the <b>unique words</b> in the document,
        where each word is associated with the list of its <b>positions</b> in the document.
            </li>
    <li>You will represent this index as a <b>dictionary</b>, where the keys will be the words, and
                the values, the lists of positions
            </li>
    <li>As <b>words</b>, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, use <b>Unicode regular expressions</b>. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The <b>word positions</b> will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use the <tt>finditer()</tt> method to find the positions of the words.
                This will return you <b>match objects</b>,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
    <li>You will use the <b>pickle package</b> to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>

Below is an excerpt of the index of the `bannlyst.txt` text for the words <i>gjord</i>, <i>uppklarnande</i>, and <i>stjärnor</i>. The data is stored in a dictionary:

<pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
</pre>
where the word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at position 8617, and <i>stjärnor</i>, once at position 8641.

#### Writing a tokenizer 
 Tokenizing texts using white spaces as word delimiters is the most elementary technique. we just replace sequences of white spaces in the text with a new line, and we consider what is between two white spaces to be a word. In the program, we use the \s character class
to represent the **white space**:
<pre>
import re 
one_token_per_line = re.sub(’\s+’, ’\n’, text)
</pre>

However, this does not work perfectly where the commas are not segmented from the words.
So it failed to tokenize the punctuation. To improve it, we need to process separately the punctuation and symbols. *We identify these characters with regular expressions and we insert white spaces around them to separate them from the words*. 

It is easier and more compact to use
the Unicode **punctuation** and **symbol** classes instead: \p{P} and \p{S}, respectively. To use the Unicode classes, we need to import the regex module.
We then tokenize the text according to white spaces as in the previous section:
<pre>
import regex as re
spaced_tokens = re.sub(’([\p{S}\p{P}])’, r’ \1 ’, text)
one_token_per_line = re.sub(’\s+’, ’\n’, spaced_tokens)
</pre>
Alternatively, as shown below, we can explicitely define the **content of words**. we can extract a sequence of words, where a word is defined as a sequence of contiguous letters.

- Write a Unicode regular expression to find words defined as **sequences of letters**.

In [3]:
# Write your regex here

# Using the content, a word is defined as a sequence of contiguous letters
letters_regex = '\p{L}+'

In [4]:
re.findall(letters_regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

- Using `regex`, write `tokenize(text)` function to tokenize a text. Return their positions. 

We extract the words with the tokenizer as before but instead of `findall()`, we use `finditer()` to return the **match objects**. We use these match objects to extract the word positions. (See Book p 170)

In [5]:
# Write your code here
"""
Uses the letters to break the text into words.
Returns a list of match objects.
"""
def tokenize(text):
    wordTokens = re.finditer('\p{L}+', text)
    return wordTokens

In [6]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

[<regex.Match object; span=(0, 2), match='En'>,
 <regex.Match object; span=(3, 7), match='gång'>,
 <regex.Match object; span=(8, 12), match='hade'>,
 <regex.Match object; span=(13, 15), match='de'>,
 <regex.Match object; span=(16, 18), match='på'>,
 <regex.Match object; span=(19, 27), match='Mårbacka'>,
 <regex.Match object; span=(28, 30), match='en'>,
 <regex.Match object; span=(31, 39), match='barnpiga'>,
 <regex.Match object; span=(41, 44), match='som'>,
 <regex.Match object; span=(45, 50), match='hette'>,
 <regex.Match object; span=(51, 55), match='Back'>,
 <regex.Match object; span=(56, 61), match='Kajsa'>]

#### Extracting indices

- Write a `text_to_idx(words)` function to extract the indices from the list of tokens (words). Return a **dictionary**, where the keys will be the tokens (words), and the values a list of positions.

*This will return us match objects, where we get the matches and the positions with the group() and start() methods.*

In [7]:
# Write your code here
"""
Builds an index from a list of match objects.
"""
def text_to_idx(words):
    word_idx = {}
    # get the matches and the positions with the group() and start() methods
    for word in words:
        try:
            word_idx[word.group()].append(word.start())
        except:
            word_idx[word.group()] = [word.start()]
    return word_idx

In [8]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 28],
 'gång': [3],
 'hade': [8],
 'de': [13],
 'på': [16],
 'mårbacka': [19],
 'barnpiga': [31],
 'som': [41],
 'hette': [45],
 'back': [51],
 'kajsa': [56]}

#### Reading one file

- Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`

In [9]:
# Write your code here
#marbackafile = open("Selma/marbacka.txt", "r", encoding="utf8")
marbackafile = open("Selma\marbacka.txt", encoding='utf-8')
marbackatxt = marbackafile.read()
#marbackafile.close()

marbaTokens = tokenize(marbackatxt.lower().strip())
#marbaTokens = tokenize(marbackatxt.lower())
idx = text_to_idx(marbaTokens)


In [10]:
idx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

#### Saving the index

- Save your index in a file so that you can reuse it. Use the **pickle module**.

In [11]:
# Write your code here
pickle.dump(idx, open("savedMarbackaIdx.p", "wb"))

In [12]:
# Write your code here
savedidx = pickle.load(open("savedMarbackaIdx.p", "rb"))

In [13]:
savedidx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

### Reading the content of a folder <a name= "t3"/>

Write a `get_files(dir, suffix)` function that reads all the files in a folder with a specific `suffix` (txt). You will need the Python `os` package, see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You will return the file names in a list.

You can reuse this function:

In [14]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [15]:
# Write your code here
SelmaFiles = get_files('Selma', '.txt')
SelmaFiles

['bannlyst.txt',
 'gosta.txt',
 'herrgard.txt',
 'jerusalem.txt',
 'kejsaren.txt',
 'marbacka.txt',
 'nils.txt',
 'osynliga.txt',
 'troll.txt']

### Creating a master index <a name="t4"/>

- Complete your program with the creation of **master index**, where you will associate each word of the corpus with the files, where it occur and its positions: a posting list.

Below is an excerpt of the master index with the words *samlar* and *ände*:

In [16]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

{'samlar': {'troll.txt': [641880, 654233],
  'nils.txt': [51805, 118943],
  'osynliga.txt': [399121],
  'gosta.txt': [313784, 409998, 538165]},
 'ände': {'troll.txt': [39562, 650112],
  'kejsaren.txt': [50171],
  'marbacka.txt': [370324],
  'nils.txt': [1794],
  'osynliga.txt': [272144]}}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
            313784, 409998, and 538165.

We build the master index of corpus Selma with a loop over the list of files.

In [17]:
# write your code here

corpus_files = get_files('Selma', '.txt')
#print("Selma Files: ", len(corpus_files), "\n", corpus_files)

master_index = {}

for file in corpus_files:
    filetext = open('Selma/' + file, encoding = 'utf-8').read()
    
    # list of tokens (words): match objects
    filewords = tokenize(filetext.lower().strip())
    
    # text_to_idx Returns a dictionary of words|positions
    # we get the matches and the positions with the group() and start() methods.
    file_idx = text_to_idx(filewords)
        
    for word in file_idx:
        if word in master_index:
            master_index[word][file] = file_idx[word]
            
        else:
            master_index[word] = {}
            master_index[word][file] = file_idx[word]
        

In [18]:
master_index['ände']

{'kejsaren.txt': [50171],
 'marbacka.txt': [370324],
 'nils.txt': [1794],
 'osynliga.txt': [272144],
 'troll.txt': [39562, 650112]}

In [19]:
master_index['samlar']

{'gosta.txt': [313784, 409998, 538165],
 'nils.txt': [51805, 118943],
 'osynliga.txt': [399121],
 'troll.txt': [641880, 654233]}

In [20]:
master_index['mårbacka']

{'marbacka.txt': [16,
  139,
  752,
  1700,
  2582,
  3324,
  15117,
  15404,
  27794,
  42175,
  49126,
  50407,
  52053,
  60144,
  63374,
  64910,
  67182,
  67330,
  67799,
  67824,
  69232,
  71328,
  72099,
  74147,
  74255,
  74614,
  76610,
  76884,
  77138,
  77509,
  77787,
  77936,
  78574,
  80597,
  81782,
  82003,
  84363,
  84786,
  85251,
  89837,
  97093,
  98642,
  100474,
  105063,
  105298,
  105721,
  108710,
  109133,
  112844,
  113725,
  114997,
  115583,
  115833,
  116368,
  116557,
  121896,
  124823,
  126409,
  126542,
  128758,
  130976,
  131939,
  132826,
  136914,
  137187,
  137872,
  139196,
  140721,
  142324,
  146781,
  151497,
  154335,
  155139,
  155438,
  155886,
  156405,
  158108,
  159817,
  160107,
  161158,
  162085,
  165847,
  168316,
  168528,
  169111,
  170333,
  172684,
  182047,
  182427,
  186362,
  189535,
  190999,
  191110,
  193177,
  196686,
  202552,
  206340,
  207789,
  208382,
  209874,
  210525,
  217464,
  219933,
  2213

- Save your master index in a file and read it again

In [21]:
# Write your code here
# save the master index in a file using pickle package
pickle.dump(master_index, open("savedMasterIdx.p", "wb"))

In [22]:
# read again the saved master index
saved_master_index = pickle.load(open("savedMasterIdx.p", "rb"))

In [23]:
saved_master_index['samlar']

{'gosta.txt': [313784, 409998, 538165],
 'nils.txt': [51805, 118943],
 'osynliga.txt': [399121],
 'troll.txt': [641880, 654233]}

#### Concordances

- Write a `concordance(word, master_index, window)` function to extract the concordances of a `word` within a `window` of characters.

Concordances are used to read **occurrences** of the given terms (or expression) in their respective context.

In [24]:
# Write your code here
def concordance(word, master_index, window):
    concord = {}
    if word in master_index:
        
        for filename in master_index[word]:
            #print(filename)
            try:
                filetext = open('Selma/' + filename, encoding='utf-8').read()
                concord[filename] = ""
            except:
                print('Could not open file', filename)
            
            for i in master_index[word][filename]:
            #for i in master_index[word]:
                if filename in concord:
                    concord[filename] += "\n\t" + filetext[(i-window):(i+window)].replace("\n", ' ')
                else:
                    concord[filename] = []
                    concord.append(filetext[(i-window):(i+window)].replace("\n", ' '))
                
        for file in concord:
                print(file, concord.get(file))
        
                       
            # spaces match tabs and newlines
#            pattern = re.sub(' ', '\\s+', word)
            # Replaces newline with spaces in the text
#            text = re.sub('\s+', ' ', filetext)
            
#            concord = "(.{{0,{width}}}{pattern}.{{0,{width}}})".format(pattern=pattern, width=window)
            #print(i.group(1))
#            for match in re.finditer(concord, text):
#                print("\t", match.group(1))
            
        

In [25]:
concordance('samlar', master_index, 25)

gosta.txt 
	om ligger nära Borg, och samlar ihop ett litet mid
	lika förstämda.  Men hon samlar upp allt detta som
	n ensam i livet.  Därmed samlar han korten tillhop
nils.txt 
	 bara, att du i all hast samlar ihop så mycket bos
	ar stannat hemma, och nu samlar de sig för att int
osynliga.txt 
	 till höger i kärran och samlar just ihop tömmarna
troll.txt 
	en örtkunnig läkare, som samlar in markens växter 
	älper dem, och medan hon samlar och handlar för de


### Representing Documents with tf-idf <a name="t5"/>

Once you have created the index, you will represent each document in your corpus as a **dictionary**. The **keys** of these dictionaries will be the words and you will define the **value** of a word with the tf-idf metric: 
1. Read the description of the tf-idf measure on Wikipedia (<a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf">https://en.wikipedia.org/wiki/Tf-idf</a>)
2. After reading the description, you probably realized that there are multiple definitions of tf-idf. In this assignment, 
 * Tf will be the **relative frequency** of the term in the document and 
 * idf, the **logarithm base 10** of the inverse document frequency.
        
You have below the **tf-idf values** for a few words. In our example, the word <i>gås</i> has the value 0 in bannlyst.txt and the value 0.000101001964 in nils.txt

<pre>
troll.txt
	känna	 0.0
	gås	 0.0
	nils	 2.148161748868631e-06
	et	 0.0
kejsaren.txt
	känna	 0.0
	gås	 0.0
	nils	 8.08284798629935e-06
	et	 8.273225429362848e-05
marbacka.txt
	känna	 0.0
	gås	 0.0
	nils	 7.582276564686669e-06
	et	 9.70107989686256e-06
herrgard.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
nils.txt
	känna	 0.0
	gås	 0.00010100196417506702
	nils	 0.00010164426900380124
	et	 0.0
osynliga.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
jerusalem.txt
	känna	 0.0
	gås	 0.0
	nils	 4.968292117670952e-06
	et	 0.0
bannlyst.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
gosta.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
</pre>

Conceptually, the tf-idf representation is a **vector**. In your program, you will keep this idea and use all the words in the corpus as keys: *Each dictionary will include all the words of the corpus as keys*. The value of the key is then possibly 0, meaning that the word is not in the document or is in all the documents as for the word `nils` in `gosta.tx`. 

As further work, you may think of optimizing this part.

In [26]:
word_dicts = {}
word_counts = {}

for wordkey in master_index:
    #print(wordkey)
    #word_dicts.update(dict(map(lambda d: (d[0], len(d[1])), master_index[wordkey].items())))
    word_dicts = dict(map(lambda d: (d[0], len(d[1])), master_index[wordkey].items()))
    for doc in word_dicts:
        if not doc in word_counts:
            word_counts[doc] = word_dicts[doc]
        else:
            word_counts[doc] += word_dicts[doc]
    #print(word_dicts)
#word_dicts
word_counts
#master_index['glömden']

{'bannlyst.txt': 76079,
 'gosta.txt': 129143,
 'herrgard.txt': 33665,
 'jerusalem.txt': 154141,
 'kejsaren.txt': 63164,
 'marbacka.txt': 67334,
 'nils.txt': 198403,
 'osynliga.txt': 82723,
 'troll.txt': 118833}

In [27]:
# Write your code here
# tf-idf representation as a dictionary/vector
tfidf_dict = {}

for wordkey in master_index:
    #print(wordkey)
    for file in master_index[wordkey]:
        # list of tokens (words): match objects
        #filewords = tokenize(filetext.lower().strip())
        
        #word_dicts = dict(map(lambda d: (d[0], len(d[1])), master_index[wordkey].items()))
        
        
        filesize = word_counts[file]
        #print(file, ":\t",filesize)
        
        # Term frequency
        tf = len(master_index[wordkey][file]) / filesize
        #print("tf: ", tf)
        
        # calculate idf value
        # N is the total number of documents in the corpus
        N = len(corpus_files)
        # nj is the number of documents where term j occurs
        nj = len(master_index[wordkey])
        idf = math.log10(N / nj)
        #print("idf: ", idf)
        
        tfxidf = tf * idf
        if file in tfidf_dict:
            tfidf_dict[file][wordkey] = tfxidf
        else:
            tfidf_dict[file] = {}
            tfidf_dict[file][wordkey] = 0
            
tfidf = tfidf_dict
#tfidf

In [28]:
tfidf['troll.txt']['känna']

0.0

In [29]:
tfidf['troll.txt']['nils']

2.1481617488686316e-06

### Comparing Documents <a name="t6"/>

Using the **cosine similarity**, compare all the pairs of documents with their tf-idf representation and present your results in a table. You will include this table in your report.

#### Cosine similarity

- Write a function computing the cosine similarity between two documents

In [30]:
# Write your code here
def cos_sim(doc1, doc2):
    # The intersection() method returns a set that 
    # contains the similarity between two or more sets.
    sim_set = set(doc1.keys()).intersection(set(doc2.keys()))
    
    # implement the formula of cosine similarity 
    # numerator values
    num = sum(doc1[key] * doc2[key] for key in sim_set)
    
    # denominator values
    norm_doc1 = math.sqrt(sum(doc1[w]*doc1[w] for w in doc1.keys()))
    norm_doc2 = math.sqrt(sum(doc2[w]*doc2[w] for w in doc2.keys()))
    denom = norm_doc1 * norm_doc2
    return num / denom


#### Similarity matrix

- Compute the similarity matrix between the documents of the corpus. While computing the similarities, you will record the two most similar documents that you will call `most_sim_doc1` and `most_sim_doc2`.

In [31]:
# Write your code here
sim_matrix = {}
cols = "\t"
for file in corpus_files:
    cols += file + " "
print(cols)

for f1 in corpus_files:
    rows = f1
    for f2 in corpus_files:
        if not f1 in sim_matrix:
            sim_matrix[f1] = {}
        sim_matrix[f1][f2] = cos_sim(tfidf[f1], tfidf[f2])
        rows += "\t" + str(round(cos_sim(tfidf[f1], tfidf[f2]), 4))
        
    print(rows)
    

	bannlyst.txt gosta.txt herrgard.txt jerusalem.txt kejsaren.txt marbacka.txt nils.txt osynliga.txt troll.txt 
bannlyst.txt	1.0	0.049	0.0009	0.0065	0.024	0.0368	0.051	0.0521	0.0886
gosta.txt	0.049	1.0	0.0031	0.0043	0.048	0.0802	0.1048	0.1248	0.1957
herrgard.txt	0.0009	0.0031	1.0	0.3707	0.0007	0.0036	0.0051	0.0048	0.0041
jerusalem.txt	0.0065	0.0043	0.3707	1.0	0.0018	0.0049	0.0045	0.0283	0.0071
kejsaren.txt	0.024	0.048	0.0007	0.0018	1.0	0.0711	0.0497	0.0511	0.1813
marbacka.txt	0.0368	0.0802	0.0036	0.0049	0.0711	1.0	0.0847	0.0932	0.1472
nils.txt	0.051	0.1048	0.0051	0.0045	0.0497	0.0847	1.0	0.1106	0.1885
osynliga.txt	0.0521	0.1248	0.0048	0.0283	0.0511	0.0932	0.1106	1.0	0.1926
troll.txt	0.0886	0.1957	0.0041	0.0071	0.1813	0.1472	0.1885	0.1926	1.0


Compute the similarity matrix - second method using **array of 2 dimensions**

sim_list = []
for i in range(len(corpus_files)):
#for i in corpus_files:
    sim_list = sim_list + [[]]
    for j in range(len(corpus_files)):
    #for j in corpus_files:
        sim_list[i] = sim_list[i] + [0]
        f1 = corpus_files[i]
        #print("f1: ", f1)
        f2 = corpus_files[j]
        #print("f2: ", f2)
        #simList[i][j] = cos_sim(tfidf[corpus_files[i]],tfidf[corpus_files[j]])
        sim_list[i][j] = cos_sim(tfidf[f1],tfidf[f2])
        #sim_list[i][j] = round(cos_sim(tfidf[f1], tfidf[f2]), 4)
        #print(sim_list[i][j])
#print(sim_list)
sim_matrix = numpy.matrix(sim_list)

#print (sim_matrix)


- Give the name of the two novels that are the most similar.

In [32]:
#diag_filtered = filter(lambda f: f[0] != f[1], ((i,j) for i, j in range(len(corpus_files))) )
#word_dicts = dict(map(lambda d: (d[0], len(d[1])), master_index[wordkey].items()))
diag_filtered = {}
max_sim = {}
for file in sim_matrix:
    #print(list(sim_matrix[file].values()))
    #print(list(sim_matrix[file].keys()))
    #fileXtfidf_dict = dict(sim_matrix[file].items())
    
    # filter away the diagonal elements
    diag_filtered[file] = dict(filter(lambda f: file != f[0], sim_matrix[file].items() ))
    
    fileXtfidf_dict = dict(diag_filtered[file].items())
    maxvalue = max(map(lambda f: (f[1]), diag_filtered[file].items()))
    #max_sim[file] = max(map(lambda f: (f[1]), diag_filtered[file].items()))
    
    for key in list(fileXtfidf_dict.keys()):
        if fileXtfidf_dict.get(key) == maxvalue:
            max_sim[file] = {key : maxvalue}
    #print(max_sim[file])
    #max_sim[file] = max(filter(lambda f: (f[1]), diag_filtered[file].items()))
    #X = fileXtfidf_dict.get(max_sim[file])
    #print(X)
#print(" Similarity list \n", max_sim)
#max_sim.values()
#for k in list(sim_matrix[file].keys()):
#similar = max(map(lambda f: (f[1]), max_sim.values()))
#print(similar)

In [33]:
def get_all_values(nested_dict, allvalues, keykeys):
        
    for keykey, value in nested_dict.items():
    #for value in nested_dict.items():
        if type(value) is dict:
            #allvalues = []
            get_all_values(value, allvalues, keykeys)
            #allvalues = []
        else:
            #print(keykey, ":", value)
            keykeys.append(keykey)
            allvalues.append(value)
    #print("all values: ", allvalues)
    
    most = max(allvalues)
    idx = allvalues.index(most)
    return keykeys, allvalues, most, idx       

In [34]:
k = list(max_sim.keys())
#v = list(max_sim.values())
massimi = []
innerKeys = []
innerDocs, maxlist, most, idx = get_all_values(max_sim, massimi, innerKeys)
#print("all keys: ", innerDocs)
most_sim_doc1 = k[idx]
most_sim_doc2 = innerDocs[idx]
max_similarity = most

#innerDict = v[idx]
#print(innerDict)
#allvalues = get_all_values(max_sim)
#print("\n file1", most_sim_doc1)
#print("\n file2", most_sim_doc2)
#print(most, " at index: ", idx)


In [35]:
print("Most similar:", most_sim_doc1, most_sim_doc2, "Similarity:", max_similarity)

Most similar: herrgard.txt jerusalem.txt Similarity: 0.370689423873392


## Submission <a name="t7"/>

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [36]:
STIL_ID = ["hi8826mo-s"] # Write your stil ids
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "1-indexer_solution_HichamMohamad.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the two most similar novels.

In [37]:
ANSWER = ' '.join(sorted([most_sim_doc1, most_sim_doc2]))
ANSWER

'herrgard.txt jerusalem.txt'

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [38]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [39]:
ASSIGNMENT = 1
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [40]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=False)


# from IPython.display import display, JSON
res.json()

C:\Users\hashu\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'msg': None,
 'status': 'correct',
 'signature': '62721125e82d86bd3aa72c8dd7badde1f74e81f298a90850c7e0a840db1e737c4408d3e80577074bf38362124265d5350d5fac0b09e644061c4a09a5abe34963',
 'submission_id': 'f8a956e1-37c2-42b3-8e82-cf7d62cf9838'}

Check the `status` and be sure it is `correct`. If not, revise your code; verify that you obtained intermediate results identical to those in the notebook; and resubmit your notebook. You can submit multiple times.

<h2>Reading</h2> <a name="t8"/>

Now you are done, it is time to write your **individual report**. You will describe the indexer and comment the results.

You will also read the text: <i>Challenges in Building Large-Scale Information Retrieval Systems</i> about the history of <a href="https://research.google.com/people/jeff/WSDM09-keynote.pdf">Google indexing</a> by <a href="https://research.google.com/pubs/jeff.html">Jeff Dean</a>.

In your report, you will tell how your index encoding is related to what **Google** did. You must identify the slide where you have the most similar indexing technique and write the slide number in your report.